In [3]:
import numpy as np
import os
import cv2
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint

# Mount Drive (Colab)
from google.colab import drive
drive.mount('/content/drive')

# Dataset path
DATASET_PATH = "/content/drive/MyDrive/Crop Identification/Agricultural-crops"
MODEL_SAVE_PATH = "/content/drive/MyDrive/Crop Identification/models"

os.makedirs(MODEL_SAVE_PATH, exist_ok=True)

# Load dataset paths
data = []
for label in os.listdir(DATASET_PATH):
    folder = os.path.join(DATASET_PATH, label)
    for img_name in os.listdir(folder):
        data.append([label, os.path.join(folder, img_name)])

df = pd.DataFrame(data, columns=["label", "path"])

# Read images
def read_image(path):
    img = cv2.imread(path)
    img = cv2.resize(img, (224, 224))
    return img

images = np.array([read_image(p) for p in df["path"]], dtype="float32")

# Encode labels
le = LabelEncoder()
labels = le.fit_transform(df["label"])

# Train-test split
train_images, test_images, train_labels, test_labels = train_test_split(
    images, labels, test_size=0.25, random_state=42
)

# Load ResNet50
base_model = keras.applications.ResNet50(
    weights="imagenet",
    include_top=False,
    input_shape=(224,224,3)
)

base_model.trainable = False

model = keras.Sequential([
    base_model,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(1024, activation='relu'),
    keras.layers.Dense(30, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='SparseCategoricalCrossentropy',
    metrics=['accuracy']
)

# Save model after every epoch
checkpoint = ModelCheckpoint(
    filepath=MODEL_SAVE_PATH + "/model_epoch_{epoch:02d}.h5",
    save_freq='epoch',
    save_best_only=False
)

# Train
model.fit(
    train_images,
    train_labels,
    epochs=10,
    batch_size=32,
    validation_data=(test_images, test_labels),
    callbacks=[checkpoint]
)

# Save final model
model.save(MODEL_SAVE_PATH + "/final_model.h5")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.2175 - loss: 3.6518

20/20 ━━━━━━━━━━━━━━━━━━━━ 152s 7s/step - accuracy: 0.2250 - loss: 3.6060 - val_accuracy: 0.5913 - val_loss: 1.3900
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.7876 - loss: 0.6576

20/20 ━━━━━━━━━━━━━━━━━━━━ 133s 7s/step - accuracy: 0.7902 - loss: 0.6522 - val_accuracy: 0.7115 - val_loss: 1.0376
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.9591 - loss: 0.1789

20/20 ━━━━━━━━━━━━━━━━━━━━ 139s 7s/step - accuracy: 0.9590 - loss: 0.1789 - val_accuracy: 0.7163 - val_loss: 1.0052
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.9939 - loss: 0.0704

20/20 ━━━━━━━━━━━━━━━━━━━━ 143s 7s/step - accuracy: 0.9937 - loss: 0.0701 - val_accuracy: 0.7067 - val_loss: 1.0041
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.9982 - loss: 0.0264

...

20/20 ━━━━━━━━━━━━━━━━━━━━ 147s 7s/step - accuracy: 1.0000 - loss: 0.0110 - val_accuracy: 0.7260 - val_loss: 0.9505
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 1.0000 - loss: 0.0083

20/20 ━━━━━━━━━━━━━━━━━━━━ 132s 7s/step - accuracy: 1.0000 - loss: 0.0083 - val_accuracy: 0.7212 - val_loss: 0.9640
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 1.0000 - loss: 0.0062

20/20 ━━━━━━━━━━━━━━━━━━━━ 141s 7s/step - accuracy: 1.0000 - loss: 0.0062 - val_accuracy: 0.7260 - val_loss: 0.9890
Epoch 9/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 1.0000 - loss: 0.0047

20/20 ━━━━━━━━━━━━━━━━━━━━ 141s 7s/step - accuracy: 1.0000 - loss: 0.0047 - val_accuracy: 0.7404 - val_loss: 0.9493
Epoch 10/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 1.0000 - loss: 0.0048

20/20 ━━━━━━━━━━━━━━━━━━━━ 141s 7s/step - accuracy: 1.0000 - loss: 0.0048 - val_accuracy: 0.7404 - val_loss: 0.9626
